In [ ]:
from typing import Optional

from autogen import ConversableAgent, LLMConfig
from autogen.events.agent_events import ExecutedFunctionEvent
from autogen.io.processors.base import EventProcessorProtocol
from autogen.io.processors.console_event_processor import ConsoleEventProcessor
from autogen.io.run_response import RunResponseProtocol
from autogen.tools import tool

In [ ]:
llm_config = LLMConfig(api_type="openai", model="gpt-4o-mini")
agent = ConversableAgent(
    name="agent",
    llm_config=llm_config,
)


@tool(description="List files and folders")
def list_files(
    folder_name: str,
) -> str:
    raise NotImplementedError("This function is not implemented yet.")


list_files.register_for_llm(agent)

In [ ]:
def custom_response_process(
    response: RunResponseProtocol,
    processor: Optional[EventProcessorProtocol] = None,
    fail_on_error_messages_list: Optional[list[str]] = None,
) -> None:
    processor = processor or ConsoleEventProcessor()
    for event in response.events:
        if (
            fail_on_error_messages_list
            and isinstance(event, ExecutedFunctionEvent)
            and not event.content.is_exec_success
            and any(error_msg in event.content.content for error_msg in fail_on_error_messages_list)
        ):
            raise RuntimeError(f"Function execution failed: {event.content}")
        processor.process_event(event)


# use this instead of the default response.process() to handle custom error messages

In [ ]:
response = agent.run(
    message="List all files and folders in the 'root' folder",
    tools=agent.tools,
    user_input=False,
    max_turns=3,
)

fail_on_error_messages_list = ["Some other error message"]
# The tool will raise NotImplementedError("This function is not implemented yet.") and we won't kill the process
custom_response_process(response=response, fail_on_error_messages_list=fail_on_error_messages_list)

In [ ]:
response = agent.run(
    message="List all files and folders in the 'root' folder",
    tools=agent.tools,
    user_input=False,
    max_turns=3,
)


fail_on_error_messages_list = ["This function is not implemented yet."]
# The tool will raise NotImplementedError("This function is not implemented yet.") and we WILL kill the process
custom_response_process(response=response, fail_on_error_messages_list=fail_on_error_messages_list)